# Dice Game

## Rules

You roll 5 dice. Each die is worth the number on the face, except for 3, which is worth zero.

In [1]:
def worth(die):
    return 0 if die == 3 else die

assert worth(1) == 1
assert worth(2) == 2
assert worth(3) == 0 # 3's are worth zero
assert worth(4) == 4
assert worth(5) == 5
assert worth(6) == 6

The entire roll is worth the sum of each die's worth.

In [2]:
def worth_roll(dice):
    return sum([worth(die) for die in dice])

assert worth_roll([1]) == 1
assert worth_roll([3, 3]) == 0
assert worth_roll([3, 3, 5]) == 5
assert worth_roll([1, 3, 3, 4, 5]) == 10

You are trying to get the lowest roll possible. You must choose at least 1 die from every roll, but you have the option of choosing up to all the dice you just rolled. Everything else will be re-rolled.

In [3]:
from collections import namedtuple
import random

def sorted_dice(dice):
    return sorted(dice, key=worth)

def choose_n(dice, n):
    assert 1 <= n <= len(dice), f"You must choose at least 1 and at most {len(dice)} dice, but you chose {n}"
    return sorted_dice(dice)[:n]

assert choose_n([1, 3, 3, 4, 5], 1) == [3]
assert choose_n([1, 3, 3, 4, 5], 2) == [3, 3]
assert choose_n([1, 3, 3, 4, 5], 3) == [3, 3, 1]
assert choose_n([1, 3, 3, 4, 5], 4) == [3, 3, 1, 4]
assert choose_n([1, 3, 3, 4, 5], 5) == [3, 3, 1, 4, 5]


GameConfig = namedtuple('GameConfig', ['num_dice', 'dice_faces'])
DEFAULT_CONFIG = GameConfig(num_dice = 5, dice_faces = 6)


class TurnState:
    def __init__(self, config):
        """num_dice and dice_faces allow for playing with a different number of dice and faces"""
        self.finalized_dice = []
        self.last_roll = None
        self.config = config

    def remaining_dice(self):
        return self.config.num_dice - len(self.finalized_dice)

    def is_over(self):
        return self.remaining_dice() == 0
    
    def score(self):
        return worth_roll(self.finalized_dice)

    def roll(self, log=False):
        self.last_roll = [random.randint(1, self.config.dice_faces) for _ in range(self.remaining_dice())]
        return self.last_roll

    def choose(self, n):
        self.finalized_dice += choose_n(self.last_roll, n)

test_turn = TurnState(DEFAULT_CONFIG)
print(f"We begin by rolling all 5: {test_turn.roll()}")
test_turn.choose(2)
assert test_turn.remaining_dice() == 3
assert not test_turn.is_over()
print(f"To demonstrate how this works (not what is good strategy), we pick the first two. So we have the dice {test_turn.finalized_dice}")
print(f"Then we re-roll the remaining 3 dice: {test_turn.roll()}")
test_turn.choose(1)
assert test_turn.remaining_dice() == 2
assert not test_turn.is_over()
print(f"We pick the first one and add it to our existing collection of finalized dice rolls: {test_turn.finalized_dice}")
print(f"Then we roll the remaining two dice that we didn't pick: {test_turn.roll()}")
test_turn.choose(2)
assert test_turn.remaining_dice() == 0
assert test_turn.is_over()
print(f"Let's say we pick both the final two. Now we have {test_turn.finalized_dice} and the turn is over with a final score of {test_turn.score()}")

We begin by rolling all 5: [2, 5, 5, 4, 2]
To demonstrate how this works (not what is good strategy), we pick the first two. So we have the dice [2, 2]
Then we re-roll the remaining 3 dice: [4, 2, 2]
We pick the first one and add it to our existing collection of finalized dice rolls: [2, 2, 2]
Then we roll the remaining two dice that we didn't pick: [1, 4]
Let's say we pick both the final two. Now we have [2, 2, 2, 1, 4] and the turn is over with a final score of 11


Every player keeps rolling until their turn is finished. The player with the lowest final score wins the round.

Here's a full round played out as a demonstration:

In [4]:
import sys

LOG_GAMEPLAY = False


def log(output):
    if LOG_GAMEPLAY:
        print(output)


class RoundState:
    """The current state of the round being played"""

    def __init__(self, players, config):
        self.lowest_score = sys.maxsize  # lowest scoring player(s) win
        self.winning_players = []
        self.turns_so_far = 0
        self.players = players
        self.config = config
        
        log(f"Starting round with {self.num_players()} players")

    def num_players(self):
        return len(self.players)
    
    def players_remaining(self):
        """How many players have yet to play in this round"""
        return self.num_players() - self.turns_so_far
        
    def record_score(self, n, player):
        if n < self.lowest_score:
            self.lowest_score = n
            self.winning_players = [player]
            log(f"{player.name} is now winning with a score of {n}")
        elif n == self.lowest_score:
            self.winning_players.append(player)
            log(f"{player.name} is tied for a win with a score of {n}")
        else:
            log(f"{player.name} lost with a score of {n}")
        
        self.turns_so_far += 1
    
    def play(self):
        for player in self.players:
            player.start_turn(self)
            player.play()
        
        winner_names = " and ".join([p.name for p in self.winning_players])
        log(f"{winner_names} won with a score of {self.lowest_score}")


class Player:
    def __init__(self, name):
        self.name = name
        self.round = None
        self.turn = None

    def start_turn(self, round_state):
        self.round = round_state
        self.turn = TurnState(self.round.config)
        log(f"{self.name} starting turn")

    def choose_n(self):
        """
        This function is called after the remaining dice have been rolled. It should return how
        many of them you'd like to keep
        """
        pass

    def play(self):
        """Automatically plays the entire round"""
        while not self.turn.is_over():
            self.turn.roll()
            log(f"{self.name} rolled {self.turn.last_roll}")
            n = self.choose_n()
            self.turn.choose(n)
            log(f"{self.name} chose to keep {n} dice. Finalized dice: {self.turn.finalized_dice}. Running score: {self.turn.score()}")

        self.round.record_score(self.turn.score(), self)
    
    @classmethod
    def create(cls, n):
        """Create n instances of this kind of player"""
        return [cls(f"{cls.__name__} {i+1}") for i in range(n)]

class RandomBot(Player):    
    def choose_n(self):
        return random.randint(1, self.turn.remaining_dice())


LOG_GAMEPLAY = True
RoundState(RandomBot.create(3), DEFAULT_CONFIG).play()

Starting round with 3 players
RandomBot 1 starting turn
RandomBot 1 rolled [5, 5, 5, 3, 2]
RandomBot 1 chose to keep 5 dice. Finalized dice: [3, 2, 5, 5, 5]. Running score: 17
RandomBot 1 is now winning with a score of 17
RandomBot 2 starting turn
RandomBot 2 rolled [6, 1, 5, 3, 1]
RandomBot 2 chose to keep 5 dice. Finalized dice: [3, 1, 1, 5, 6]. Running score: 13
RandomBot 2 is now winning with a score of 13
RandomBot 3 starting turn
RandomBot 3 rolled [4, 6, 2, 2, 6]
RandomBot 3 chose to keep 5 dice. Finalized dice: [2, 2, 4, 6, 6]. Running score: 20
RandomBot 3 lost with a score of 20
RandomBot 2 won with a score of 13


## Strategy

### Keeping only 3's

An obvious improvement on the random strategy is to only ever keep 3's, unless there are no 3's, in which case the player only keeps the one lowest die.

In [5]:
class Only3sBot(Player):    
    def choose_n(self):
        return Only3sBot.choose_n_strategy(self.turn.last_roll)
    
    @staticmethod
    def choose_n_strategy(last_roll):
        num_3s = last_roll.count(3)
        return max(num_3s, 1)

    
assert Only3sBot.choose_n_strategy([1, 2, 3, 4, 6]) == 1
assert Only3sBot.choose_n_strategy([1, 2, 3, 3, 6]) == 2
assert Only3sBot.choose_n_strategy([3, 3, 3, 3, 3]) == 5
# we always have to choose at least 1 die, even if every single one is horrible
assert Only3sBot.choose_n_strategy([6, 6, 6, 6, 6]) == 1

Let's see it in action:

In [6]:
RoundState(Only3sBot.create(3), DEFAULT_CONFIG).play()

Starting round with 3 players
Only3sBot 1 starting turn
Only3sBot 1 rolled [4, 4, 2, 1, 1]
Only3sBot 1 chose to keep 1 dice. Finalized dice: [1]. Running score: 1
Only3sBot 1 rolled [1, 1, 5, 5]
Only3sBot 1 chose to keep 1 dice. Finalized dice: [1, 1]. Running score: 2
Only3sBot 1 rolled [5, 4, 3]
Only3sBot 1 chose to keep 1 dice. Finalized dice: [1, 1, 3]. Running score: 2
Only3sBot 1 rolled [6, 5]
Only3sBot 1 chose to keep 1 dice. Finalized dice: [1, 1, 3, 5]. Running score: 7
Only3sBot 1 rolled [1]
Only3sBot 1 chose to keep 1 dice. Finalized dice: [1, 1, 3, 5, 1]. Running score: 8
Only3sBot 1 is now winning with a score of 8
Only3sBot 2 starting turn
Only3sBot 2 rolled [6, 3, 1, 1, 3]
Only3sBot 2 chose to keep 2 dice. Finalized dice: [3, 3]. Running score: 0
Only3sBot 2 rolled [4, 1, 1]
Only3sBot 2 chose to keep 1 dice. Finalized dice: [3, 3, 1]. Running score: 1
Only3sBot 2 rolled [5, 1]
Only3sBot 2 chose to keep 1 dice. Finalized dice: [3, 3, 1, 1]. Running score: 2
Only3sBot 2 ro

The only way to be sure it's actually an improvement over `RandomBot` is to get them to play lots of rounds against each other, repeatedly:

In [7]:
class Game:
    def __init__(self, players, config):
        self.players = players
        self.config = config
        self.wins = { p:0 for p in players }
    
    def play(self, num_rounds):
        for _ in range(num_rounds):
            new_round = RoundState(self.players, self.config)
            new_round.play()
            for winner in new_round.winning_players:
                self.wins[winner] += 1
        
        print(f"Played {num_rounds} rounds")
        best_players = sorted(self.players, key=lambda p: self.wins[p], reverse=True)
        for player in best_players:
            print(f"{player.name} won {self.wins[player]} rounds")


LOG_GAMEPLAY = False
Game(RandomBot.create(3) + Only3sBot.create(3), DEFAULT_CONFIG).play(num_rounds=1000)

Played 1000 rounds
Only3sBot 1 won 332 rounds
Only3sBot 2 won 316 rounds
Only3sBot 3 won 313 rounds
RandomBot 2 won 84 rounds
RandomBot 1 won 80 rounds
RandomBot 3 won 75 rounds


### Even Better Strategies

As the number of players approaches infinity, someone is guaranteed to end up with all 3's, and so the only 3's strategy becomes optimal. But as the number of players approaches zero, sometimes it's worth keeping other numbers in addition to 3. For example, in the case where there's zero players left because you're the very last player, it's obviously better to stop rolling as soon as you win, even if you win with some suboptimal 1's and 2's in your roll.

But the question is, when exactly does this happen? Intuition tells us that we have to balance out the risk of ending up with a higher roll than we have now, with the risk of someone after us ending up with a lower roll than we have now.

In [8]:
class DiceCalculator:
    def __init__(self, config, debug=True):
        self.config = config
        self.debug = debug

    def _debug(self, indent, message):
        if self.debug:
            indent_str = "  " * indent
            print(indent_str + message)
    
    def _roll_key(self, roll):
        return " ".join([str(x) for x in sorted_dice(roll)])

    def _rolls(self, dice_remaining):
        if dice_remaining == 0:
            yield []
        else:
            for first_die in range(1, self.config.dice_faces + 1):
                for subsequent_dice in self._rolls(dice_remaining - 1):
                    yield [first_die] + subsequent_dice
    
    def _possible_cuts(self, roll):
        ordered_roll = sorted_dice(roll)
        sorted_values = [worth(die) for die in ordered_roll]
        for cut in range(1, len(roll) + 1):
            taken = ordered_roll[:cut]
            # sum the value of the dice we keep
            taken_value = sum(sorted_values[:cut])
            # as for the rest, only their count matters because we're rerolling
            reroll_count = len(roll) - cut
            yield taken, taken_value, reroll_count

class ExpectationCalculator(DiceCalculator):
    def __init__(self, config):
        super().__init__(config)
        self._num_cache = {}
        self._roll_cache = {}
    
    def expectation_roll(self, roll, indent=0):
        if not roll:
            return 0

        cache = self._roll_cache
        roll_key = self._roll_key(roll)
        if roll_key in cache:
            result = cache[roll_key]
            self._debug(indent, f"We already know the roll {roll} = {roll_key} expects {result}")
            return result
        
        self._debug(indent, f"Finding expected result of rolling {roll}")
        result = None
        
        decision_values = []
        for taken, taken_value, reroll_count in self._possible_cuts(roll):
            # find the expectation value of rerolling the rest of the dice
            expected_reroll_value = self.expectation(reroll_count, indent + 1)
            expected_total = taken_value + expected_reroll_value
            self._debug(
                indent + 1,
                f"Taking {taken} and rolling {reroll_count} dice expects {expected_total}"
            )
            decision_values.append(expected_total)
        # we make the decision that results in the best expected payoff
        result = min(decision_values)
        cache[roll_key] = result
        self._debug(indent, f"We now know the roll {roll} = {roll_key} expects {result}")
        return result
    
    def expectation(self, num_dice, indent=0):
        if num_dice == 0:
            return 0

        cache = self._num_cache
        if num_dice in cache:
            result = cache[num_dice]
            #self._debug(indent, f"We already know rolling {num_dice} fresh dice expects {result}")
            return result
        
        self._debug(indent, f"Finding expected result of rolling {num_dice} fresh dice")

        total = 0
        count = 0
        for roll in self._rolls(num_dice):
            total += self.expectation_roll(roll, indent + 1)
            count += 1
        result = total / count
        
        cache[num_dice] = result
        self._debug(indent, f"We now know rolling {num_dice} fresh dice has an expected score of {result}")
        return result


expectations = ExpectationCalculator(DEFAULT_CONFIG)
assert expectations.expectation(num_dice=0) == 0  # no dice contributes no score
assert expectations.expectation(num_dice=1) == 3  # sum 1 - 6, minus 3, divide by 6

Finding expected result of rolling 1 fresh dice
  Finding expected result of rolling [1]
    Taking [1] and rolling 0 dice expects 1
  We now know the roll [1] = 1 expects 1
  Finding expected result of rolling [2]
    Taking [2] and rolling 0 dice expects 2
  We now know the roll [2] = 2 expects 2
  Finding expected result of rolling [3]
    Taking [3] and rolling 0 dice expects 0
  We now know the roll [3] = 3 expects 0
  Finding expected result of rolling [4]
    Taking [4] and rolling 0 dice expects 4
  We now know the roll [4] = 4 expects 4
  Finding expected result of rolling [5]
    Taking [5] and rolling 0 dice expects 5
  We now know the roll [5] = 5 expects 5
  Finding expected result of rolling [6]
    Taking [6] and rolling 0 dice expects 6
  We now know the roll [6] = 6 expects 6
We now know rolling 1 fresh dice has an expected score of 3.0


We can see that if you're just trying to minimize your expected total value (e.g. when you're setting the record and there's people after you), it can make sense to take a 1 die, but not a 2 die. The same goes for taking multiple 1 die:

In [9]:
expectations.debug = False
assert expectations.expectation_roll([1, 6]) == 4  # 1 + expected value of rolling 1 die
assert round(expectations.expectation(2), 2) == 4.39
assert expectations.expectation_roll([1, 1, 6]) == 5
assert round(expectations.expectation(3), 2) == 5.23

However, the game punishes you just as much for losing by a little as it does for losing by a lot. So if you're not the record setter, then you really only care about maximizing the likelihood of beating someone else's low score, not minimizing your own expected score. For example, to beat a score of 2 with potentially 2 remaining rolls:

In [10]:
def percentage(p):
    return "{:.2f}%".format(round(p, 4) * 100)


class ProbabilityCalculator(DiceCalculator):
    def __init__(self, config):
        super().__init__(config)
        self._p_num_cache = {}
        self._p_roll_cache = {}

    def p_beat_with_roll(self, goal, roll, indent=0):
        if goal <= 0:  # impossible to beat a goal of zero
            return 0
        if not roll:  # impossible to lose with no more rolls left to go
            return 1
        
        cache = self._p_roll_cache
        ordered_roll_str = self._roll_key(roll)
        roll_key = f"{ordered_roll_str} < {goal}"
        if roll_key in cache:
            result = cache[roll_key]
            self._debug(indent, f"We already know the roll {roll} => {roll_key} has {result} chance to win")
            return result
        
        self._debug(indent, f"Finding probability of winning {roll} => {roll_key}")
        decision_values = []
        for taken, taken_value, reroll_count in self._possible_cuts(roll):
            # find the expectation value of rerolling under the new taken value
            p_beat_on_reroll = self.p_beat(goal - taken_value, reroll_count, indent + 1)
            self._debug(
                indent + 1,
                f"Taking {taken} and rolling {reroll_count} dice has {p_beat_on_reroll} chance to win"
            )
            decision_values.append(p_beat_on_reroll)
        
        result = max(decision_values)
        cache[roll_key] = result
        self._debug(indent, f"We now know the roll {roll} => {roll_key} has {result} chance to win")
        return result
    
    def p_beat(self, goal, num_dice, indent=0):
        if goal <= 0:
            return 0
        if num_dice == 0:
            return 1
        
        cache = self._p_num_cache
        num_key = f"{num_dice} fresh dice < {goal}"
        if num_key in cache:
            result = cache[num_key]
            return result
        
        self._debug(indent, f"Finding probability of winning {num_key}")

        total = 0
        count = 0
        for roll in self._rolls(num_dice):
            total += self.p_beat_with_roll(goal, roll, indent + 1)
            count += 1
        result = total / count
        
        cache[num_key] = result
        self._debug(indent, f"We now know probability of winning {num_key} is {result}")
        return result

probability = ProbabilityCalculator(DEFAULT_CONFIG)
assert percentage(probability.p_beat(2, 1)) == "33.33%"
assert percentage(probability.p_beat_with_roll(3, [3, 1, 6])) == "35.65%"

Finding probability of winning 1 fresh dice < 2
  Finding probability of winning [1] => 1 < 2
    Taking [1] and rolling 0 dice has 1 chance to win
  We now know the roll [1] => 1 < 2 has 1 chance to win
  Finding probability of winning [2] => 2 < 2
    Taking [2] and rolling 0 dice has 0 chance to win
  We now know the roll [2] => 2 < 2 has 0 chance to win
  Finding probability of winning [3] => 3 < 2
    Taking [3] and rolling 0 dice has 1 chance to win
  We now know the roll [3] => 3 < 2 has 1 chance to win
  Finding probability of winning [4] => 4 < 2
    Taking [4] and rolling 0 dice has 0 chance to win
  We now know the roll [4] => 4 < 2 has 0 chance to win
  Finding probability of winning [5] => 5 < 2
    Taking [5] and rolling 0 dice has 0 chance to win
  We now know the roll [5] => 5 < 2 has 0 chance to win
  Finding probability of winning [6] => 6 < 2
    Taking [6] and rolling 0 dice has 0 chance to win
  We now know the roll [6] => 6 < 2 has 0 chance to win
We now know prob

Does this result generalize? As it turns out, you should actually only reroll both if you're within 3 of the number you're trying to beat. Otherwise, with a large enough safety margin, you're better off on average just keeping the 1 and rerolling the other die:

In [11]:
probability.debug = False
# sanity check: with 2 fresh dice rolls, there's 36 possibilities. 1 of them involves beating 1
# immediately (the 3, 3 combo). 10 of them (first roll 3, second roll not 3 -- and vice versa)
# allow for a second turn, which absolutely must be a 3. There's a 1/6th chance of that happening
# on the second roll, so that's 10 / 6 = 1.666... weighted possibility for the single 3's to
# succeed. Add that to the 1 from the 3, 3 combo and you have 2.666... Divide by 36 for all
# possibilities to get 0.07407...
assert percentage(probability.p_beat(1, 2)) == "7.41%"

for goal in range(8, 0, -1):
    take_one = percentage(probability.p_beat(goal - 1, 1))
    dont_take = percentage(probability.p_beat(goal, 2))
    print(f"Beating {goal} by taking the 1: {take_one}; beating {goal} by rerolling both: {dont_take}")

Beating 8 by taking the 1: 100.00%; beating 8 by rerolling both: 86.11%
Beating 7 by taking the 1: 83.33%; beating 7 by rerolling both: 80.56%
Beating 6 by taking the 1: 66.67%; beating 6 by rerolling both: 68.06%
Beating 5 by taking the 1: 50.00%; beating 5 by rerolling both: 56.94%
Beating 4 by taking the 1: 50.00%; beating 4 by rerolling both: 45.37%
Beating 3 by taking the 1: 33.33%; beating 3 by rerolling both: 35.65%
Beating 2 by taking the 1: 16.67%; beating 2 by rerolling both: 19.91%
Beating 1 by taking the 1: 0.00%; beating 1 by rerolling both: 7.41%


Let's look at the chance of beating a number with a fresh turn (all 5 rolls). Turns out getting all 3's is not that crazy impossible.

In [12]:
for goal in range(10, 0, -1):
    chance = percentage(probability.p_beat(goal, 5))
    print(f"Chance of beating {goal} with a fresh turn: {chance}")

Chance of beating 10 with a fresh turn: 87.64%
Chance of beating 9 with a fresh turn: 81.16%
Chance of beating 8 with a fresh turn: 72.45%
Chance of beating 7 with a fresh turn: 61.89%
Chance of beating 6 with a fresh turn: 49.93%
Chance of beating 5 with a fresh turn: 38.23%
Chance of beating 4 with a fresh turn: 26.71%
Chance of beating 3 with a fresh turn: 16.21%
Chance of beating 2 with a fresh turn: 7.30%
Chance of beating 1 with a fresh turn: 2.11%


So with that in mind, if we've already beat the previous record, what score should we aim for in order to beat the players coming after us? Turns out, it really depends on the number of players there are. With 4 players, you have a decent chance of winning with just a score of 4, but with 8 players, you'd probably want to go for at most 3.

(In this table, $s$ represents the score you're ending your turn with and $p$ represents the number of players going after you. If there's no players because you're the last one, then of course you will always win.)

In [13]:
class WinningCalculator(DiceCalculator):
    def __init__(self, config, probability_calculator=None):
        super().__init__(config)
        self.probability_calculator = probability_calculator or ProbabilityCalculator(config)
    
    def p_winning(self, your_score, num_players):
        p_one_player_beating_you = self.probability_calculator.p_beat(your_score, self.config.num_dice)
        p_you_beat_one_player = 1 - p_one_player_beating_you
        p_you_beat_every_player = p_you_beat_one_player ** num_players
        return p_you_beat_every_player


winning_calculator = WinningCalculator(DEFAULT_CONFIG, probability)
assert percentage(winning_calculator.p_winning(1, 1)) == "97.89%"
assert percentage(winning_calculator.p_winning(1, 2)) == "95.83%"

def print_table(table):
    for row in table:
        print("".join([cell.ljust(10) for cell in row]))


percentages_header = [f"p={p}" for p in range(9)]
s_p_table = [["s/p"] + percentages_header]
for score in range(10, -1, -1):
    percentages = [percentage(winning_calculator.p_winning(score, n_players)) for n_players in range(9)]
    s_p_table.append([f"s={score}"] + percentages)
print_table(s_p_table)

s/p       p=0       p=1       p=2       p=3       p=4       p=5       p=6       p=7       p=8       
s=10      100.00%   12.36%    1.53%     0.19%     0.02%     0.00%     0.00%     0.00%     0.00%     
s=9       100.00%   18.84%    3.55%     0.67%     0.13%     0.02%     0.00%     0.00%     0.00%     
s=8       100.00%   27.55%    7.59%     2.09%     0.58%     0.16%     0.04%     0.01%     0.00%     
s=7       100.00%   38.11%    14.52%    5.53%     2.11%     0.80%     0.31%     0.12%     0.04%     
s=6       100.00%   50.07%    25.07%    12.55%    6.28%     3.15%     1.57%     0.79%     0.39%     
s=5       100.00%   61.77%    38.15%    23.57%    14.56%    8.99%     5.55%     3.43%     2.12%     
s=4       100.00%   73.29%    53.72%    39.38%    28.86%    21.15%    15.50%    11.36%    8.33%     
s=3       100.00%   83.79%    70.21%    58.83%    49.30%    41.31%    34.61%    29.00%    24.30%    
s=2       100.00%   92.70%    85.93%    79.66%    73.84%    68.45%    63.45%    58.82%    5

The final question is, how do we decide what cut to make for a specific turn? Ideally we'd balance out the probability of not beating out the players before us with the probability of being beaten out by the players after us. For simplicity though, let's just first focus on beating out the players before us, and then optimize for the players after us.

In [14]:
class TurnCalculator(DiceCalculator):
    def __init__(self, config, winning_calculator=None):
        super().__init__(config)
        self.winning_calculator = winning_calculator or WinningCalculator(config)
        self.probability_calculator = self.winning_calculator.probability_calculator
        self._p_win_or_tie_cache = {}
        self._choose_n_cache = {}
    
    def p_win_or_tie(self, score_to_tie, score_so_far, dice_remaining, players_after, indent=0):
        if indent > 20:
            raise "Too recursive"
        if score_to_tie is not None and score_to_tie < score_so_far:
            return 0
        if dice_remaining == 0:
            chance = self.winning_calculator.p_winning(score_so_far, players_after)
            self._debug(indent, f"With a final score of {score_so_far}, our chance of beating {players_after} players after us is {chance}")
            return chance
        
        cache = self._p_win_or_tie_cache
        roll_key = f"{score_to_tie} > {score_so_far} + {dice_remaining} dice >= {players_after} players"
        if roll_key in cache:
            result = cache[roll_key]
            self._debug(indent, f"We already know probability of winning {roll_key} is {result}")
            return result
        
        self._debug(indent, f"Finding probability of winning {roll_key}")

        total = 0
        count = 0
        for roll in self._rolls(dice_remaining):
            n, new_score = self.choose_n(
                score_to_tie,
                score_so_far,
                roll,
                players_after,
                indent + 1
            )
            total += self.p_win_or_tie(
                score_to_tie,
                new_score,
                dice_remaining - n,
                players_after,
                indent + 1
            )
            count += 1
        result = total / count
        
        cache[roll_key] = result
        self._debug(indent, f"We now know probability of winning {roll_key} is {result}")
        return result
    
    def choose_n(self, score_to_tie, score_so_far, current_roll, players_after, indent=0):
        if score_to_tie is not None and score_to_tie < score_so_far:
            # impossible to win, give up now
            return len(current_roll), score_so_far + worth_roll(current_roll)
        if current_roll == 0:
            return 0, score_so_far
        
        cache = self._choose_n_cache
        ordered_roll_str = self._roll_key(current_roll)
        roll_key = f"{score_to_tie} >= {score_so_far} + {ordered_roll_str} >= {players_after} players"
        if roll_key in cache:
            result = cache[roll_key]
            self._debug(indent, f"We already know to choose {result} to tie {roll_key}")
            return result
        
        self._debug(indent, f"Making decision for {roll_key}")
        n = None
        new_score = None
        best_p = -1  # probabilities can never be zero, so any probability will override this
        for taken, taken_value, reroll_count in self._possible_cuts(current_roll):
            p_beat_on_reroll = self.p_win_or_tie(
                score_to_tie,
                score_so_far + taken_value,
                reroll_count,
                players_after,
                indent + 1
            )
            self._debug(
                indent + 1,
                f"Taking {taken} and rolling {reroll_count} dice has {p_beat_on_reroll} chance to win"
            )
            if p_beat_on_reroll > best_p:
                self._debug(indent + 1, f"{p_beat_on_reroll} > {best_p}")
                best_p = p_beat_on_reroll
                n = len(taken)
                new_score = score_so_far + taken_value
        
        result = (n, new_score)
        cache[roll_key] = result
        self._debug(indent, f"We now know to choose {result} to beat {roll_key}")
        return result
    

turn_calculator = TurnCalculator(DEFAULT_CONFIG, winning_calculator)
turn_calculator.debug = False
assert turn_calculator.p_win_or_tie(
    score_to_tie=None,
    score_so_far=5,
    dice_remaining=0,
    players_after=0
) == 1
assert turn_calculator.p_win_or_tie(
    score_to_tie=3,
    score_so_far=5,
    dice_remaining=4,
    players_after=0
) == 0
# 2.11% of getting perfect 3's, 7.30% of getting a score below 2 => 7.30 - 2.11 = 5.19% chance of
# scoring exactly 1. If you score exactly 1, there's a 91.83% chance the next 4 players won't beat
# you. That's a 2.11% chance of getting perfect 3's and winning for sure, and a
# 5.19 * 91.83% = 4.77% chance of winning with a score of exactly 1. In total, that's ~6.8%
# chance of winning
assert percentage(turn_calculator.p_win_or_tie(
    score_to_tie=1,
    score_so_far=0,
    dice_remaining=5,
    players_after=4
)) == "6.85%"

Let's validate our initial intuitions. If you're the last player and you can win right now, then there's no point in not picking everything, even if the score is horrendous, if it lets you win:

In [15]:
assert turn_calculator.choose_n(
    score_to_tie=8,
    score_so_far=0,
    current_roll=[3, 1, 3, 2, 4],
    players_after=0
) == (5, 7)

On the other hand, if there's lots of players after you, then you only pick the 3's:

In [16]:
assert turn_calculator.choose_n(
    score_to_tie=8,
    score_so_far=0,
    current_roll=[3, 1, 3, 2, 4],
    players_after=10
) == (2, 0)

Similarly, if you're the last player but the score to tie is really low, then you only pick the 3's as well:

In [17]:
assert turn_calculator.choose_n(
    score_to_tie=2,
    score_so_far=0,
    current_roll=[3, 1, 3, 2, 4],
    players_after=0
) == (2, 0)

### Putting it all together

Let's put all this in action for the most optimal bot possible:

In [18]:
class OptimalBot(Player):    
    def choose_n(self):
        score_to_tie = self.round.lowest_score if self.round.lowest_score != sys.maxsize else None
        score_so_far = worth_roll(self.turn.finalized_dice)
        current_roll = self.turn.last_roll
        # -1 because this bot is one of the remaining players
        players_after = self.round.players_remaining() - 1
        # every bot shares the same turn calculator for efficiency because they're all using the
        # same strategy
        n, _ = turn_calculator.choose_n(
            score_to_tie=score_to_tie,
            score_so_far=score_so_far,
            current_roll=current_roll,
            players_after=players_after,
        )
        return n


LOG_GAMEPLAY=True
RoundState(OptimalBot.create(3), DEFAULT_CONFIG).play()

Starting round with 3 players
OptimalBot 1 starting turn
OptimalBot 1 rolled [6, 4, 4, 1, 1]
OptimalBot 1 chose to keep 1 dice. Finalized dice: [1]. Running score: 1
OptimalBot 1 rolled [1, 5, 4, 3]
OptimalBot 1 chose to keep 1 dice. Finalized dice: [1, 3]. Running score: 1
OptimalBot 1 rolled [5, 4, 3]
OptimalBot 1 chose to keep 1 dice. Finalized dice: [1, 3, 3]. Running score: 1
OptimalBot 1 rolled [3, 4]
OptimalBot 1 chose to keep 1 dice. Finalized dice: [1, 3, 3, 3]. Running score: 1
OptimalBot 1 rolled [6]
OptimalBot 1 chose to keep 1 dice. Finalized dice: [1, 3, 3, 3, 6]. Running score: 7
OptimalBot 1 is now winning with a score of 7
OptimalBot 2 starting turn
OptimalBot 2 rolled [1, 1, 5, 2, 6]
OptimalBot 2 chose to keep 1 dice. Finalized dice: [1]. Running score: 1
OptimalBot 2 rolled [1, 5, 6, 1]
OptimalBot 2 chose to keep 1 dice. Finalized dice: [1, 1]. Running score: 2
OptimalBot 2 rolled [4, 5, 4]
OptimalBot 2 chose to keep 1 dice. Finalized dice: [1, 1, 4]. Running score: 

Make sure it handles a decreasing number of players through the round:

In [19]:
import re

def search_calculator_cache(turn_calculator, regex, keep):
    for k, (n, _) in turn_calculator._choose_n_cache.items():
        if n == keep and re.match(regex, k):
            print(f"Choose {n} for {k}")
            return k, n

# no key exists for 3 players because there's 3 players in total, so there will never be a player
# with 3 players after them
assert search_calculator_cache(turn_calculator, ".*2 players$", keep=1) is not None
assert search_calculator_cache(turn_calculator, ".*1 players$", keep=1) is not None
assert search_calculator_cache(turn_calculator, ".*0 players$", keep=1) is not None

Choose 1 for None >= 4 + 1 >= 2 players
Choose 1 for 7 >= 4 + 1 >= 1 players
Choose 1 for 8 >= 3 + 1 >= 0 players


There's only one way to know if it actually beats out the previous strategy of always only keeping 3's:

In [20]:
LOG_GAMEPLAY = False
players = [p for pairs in zip(Only3sBot.create(3), OptimalBot.create(3)) for p in pairs]
Game(players, DEFAULT_CONFIG).play(num_rounds=10000)

Played 10000 rounds
OptimalBot 3 won 2512 rounds
OptimalBot 2 won 2383 rounds
OptimalBot 1 won 2299 rounds
Only3sBot 3 won 1909 rounds
Only3sBot 1 won 1830 rounds
Only3sBot 2 won 1767 rounds


So it does! Moreover, it appears that the last player is in the best position, and only keeping 3's is already such a great rule of thumb.

Out of curiosity, does it ever make sense to voluntarily keep a 2 even when there are people after you?

In [21]:
search_calculator_cache(turn_calculator, ".*\+ 3 2.*>= 4 players$", keep=2)

Choose 2 for 9 >= 2 + 3 2 >= 4 players


('9 >= 2 + 3 2 >= 4 players', 2)

Apparently so! If there's 4 players after you, and you currently have a score of 2, but you can end your turn right now, then you should take the 2. But absolutely not if you're going to keep rolling.

In [22]:
search_calculator_cache(turn_calculator, ".*\+ 3 2 \d.*>= 1 players$", keep=2)

Sometimes (but not always), it makes sense to voluntarily keep a 1 -- even two 1's, and even if you're still rerolling after this:

In [24]:
search_calculator_cache(turn_calculator, ".*\+ 3 1 \d.*>= 1 players$", keep=2)
search_calculator_cache(turn_calculator, ".*\+ 3 1 1.*>= 1 players$", keep=3)
search_calculator_cache(turn_calculator, ".*\+ 3 1 1 \d.*>= 1 players$", keep=3)
search_calculator_cache(turn_calculator, ".*\+ 1 1 \d.*>= 1 players$", keep=2)

Choose 2 for 7 >= 2 + 3 1 4 >= 1 players
Choose 3 for 7 >= 2 + 3 1 1 >= 1 players
Choose 2 for 7 >= 1 + 1 1 4 >= 1 players


('7 >= 1 + 1 1 4 >= 1 players', 2)

## Summary

- In the situation where there's 2 dice left and one of them is a 1: While it's true that we're *slightly* more likely to not regret rerolling both dice, you should only do so if you're pressured to beat a score that's within 3 of your current one. Otherwise, it's actually better to take the 1 on average (so our intuitions for distrusting the math sometimes was right)
- Sometimes (but not always), it makes sense to even keep multiple 1's, even if you're still rolling afterwards
- 4 is a good score to aim for when you have 4 or 5 players, but we had 8 players, so it's better to aim for 3 or even 2
- There's a 2% chance of rolling all 3's, which is actually not that crazy low
- Each spot down in turn order seems to confer an approximately 2% advantage, so the 5th player to take their turn after you has ~10% advantage on you, which explains why there's the rule that the winner is supposed to move first next round, to keep it even